<a href="https://colab.research.google.com/github/kadir28566/PPEDETECTIONYOLOv8/blob/master/ppe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ================== YÜKLEMELER ==================
!pip install -q ultralytics gradio ffmpeg-python opencv-python

# ================== İMPORTLAR VE MODEL ==================
import gradio as gr
from ultralytics import YOLO
import os
import cv2
import uuid
import ffmpeg
import pandas as pd
import numpy as np

model = YOLO("best.pt")  # Eğer dosya varsa, yoksa drive'dan yükleyebilirsin

# ================== YARDIMCI FONKSİYONLAR ==================

def avi_to_mp4(input_file, output_file):
    try:
        (
            ffmpeg
            .input(input_file)
            .output(output_file, vcodec='libx264', acodec='aac')
            .overwrite_output()
            .run()
        )
        return output_file
    except ffmpeg.Error as e:
        print(f"FFmpeg Hatası: {e}")
        return input_file

def get_equipment_names(results):
    equipment_data = []
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])
            confidence = round(float(box.conf[0]), 2)
            class_name = result.names[class_id]
            equipment_data.append({
                "Class_ID": class_id,
                "Class Name": class_name,
                "Confidence": confidence
            })
    return pd.DataFrame(equipment_data)

def check_compliancy(results):
    def is_inside(box_a, box_b):
        return (
            box_b[0] >= box_a[0] and
            box_b[1] >= box_a[1] and
            box_b[2] <= box_a[2] and
            box_b[3] <= box_a[3]
        )

    persons, helmets, vests = [], [], []
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])
            xyxy = box.xyxy[0].tolist()
            label = result.names[class_id]

            if label == "person":
                persons.append(xyxy)
            elif label == "helmet":
                helmets.append(xyxy)
            elif label == "vest":
                vests.append(xyxy)

    people_info = []
    no_vest = no_helmet = no_compliance = 0
    warn_str = ""
    for p_box in persons:
        has_helmet = any(is_inside(p_box, h) for h in helmets)
        has_vest = any(is_inside(p_box, v) for v in vests)

        people_info.append({
            'Has helmet': has_helmet,
            'Has vest': has_vest,
            'Compliance': has_helmet and has_vest,
        })

        if not has_vest and not has_helmet:
            no_compliance += 1
        if not has_vest and has_helmet:
            no_vest += 1
        if has_vest and not has_helmet:
            no_helmet += 1

    if no_vest > 0:
        warn_str += f"{no_vest} person has no vest!\n"
    if no_helmet > 0:
        warn_str += f"{no_helmet} person has no helmet!\n"
    if no_compliance > 0:
        warn_str += f"{no_compliance} person has no compliance!\n"

    return pd.DataFrame(people_info), warn_str


# ================== PREDICT FONKSİYONLARI ==================

def get_predict(image):
    results = model.predict(image, save=True, conf=0.4, project="./", name="results", exist_ok=True)
    table = get_equipment_names(results)
    table2, warn_str = check_compliancy(results)
    pic_files = [f for f in os.listdir("./results/") if f.endswith((".jpg", ".png"))]
    pic_files.sort(key=lambda x: os.path.getmtime(os.path.join("./results/", x)))
    pic_path = os.path.join("./results/", pic_files[-1])
    img = cv2.imread(pic_path)
    return img[:, :, ::-1], table, table2, warn_str  # RGB formatına çevir

def get_predict_from_video(video_path):
    try:
        print(f"Video işleniyor: {video_path}")
        unique_name = f"video_results_{uuid.uuid4().hex[:8]}"
        results = model.track(
            video_path,
            save=True,
            conf=0.4,
            iou=0.5,
            tracker="bytetrack.yaml",
            project="./",
            name=unique_name,
            exist_ok=True,
            show=False
        )

        data = []
        for result in results:
            for box in result.boxes:
                track_id = int(box.id[0]) if box.id is not None else -1
                class_id = int(box.cls[0])
                conf = float(box.conf[0])
                class_name = result.names[class_id]

                data.append({
                    "Track ID": track_id,
                    "Class Name": class_name,
                    "Conf": round(conf, 2)
                })

        df = pd.DataFrame(data).drop_duplicates(subset=["Track ID", "Class Name"])
        df = df.sort_values("Conf", ascending=False).drop_duplicates(subset=["Track ID"])

        output_dir = f"./{unique_name}/"
        video_files = [f for f in os.listdir(output_dir) if f.endswith((".mp4", ".avi"))]
        if video_files:
            yolo_output_path = os.path.join(output_dir, video_files[-1])
            video = avi_to_mp4(yolo_output_path, f'{output_dir}/video.mp4')

            # Uyarı çıkar
            table2, warn_str = check_compliancy(results)
            return video, df, warn_str

        return video_path, df, "Çıktı videosu bulunamadı."

    except Exception as e:
        print(f"Hata oluştu: {e}")
        return video_path, pd.DataFrame(), "Bir hata oluştu."

# ================== GRADIO ARAYÜZÜ ==================

def detect_from_image(image):
    return get_predict(image)

def detect_from_video(video):
    return get_predict_from_video(video)

with gr.Blocks() as demo:
    gr.Markdown("# 🦺 Nesne Tespiti - Resim ve Video")

    with gr.Tab("📷 Resimden Tespit"):
        with gr.Row():
            image_input = gr.Image(type="numpy", label="Resim Yükle")
            image_output = gr.Image(label="Tespit Sonucu")

        image_button = gr.Button("Nesne Tespiti Yap")

        with gr.Row():
            table_output = gr.Dataframe(label="Tespit Tablosu")
            table2_output = gr.Dataframe(label="Uyarı Tablosu")

        warn_str = gr.Textbox(label="Warning Notes")
        image_button.click(
            fn=detect_from_image,
            inputs=image_input,
            outputs=[image_output, table_output, table2_output, warn_str]
        )

    with gr.Tab("🎥 Videodan Tespit"):
        with gr.Row():
            video_input = gr.Video(label="Video Yükle")
            video_output = gr.Video(label="Tespit Sonucu")

        with gr.Row():
            video_table_output = gr.Dataframe(label="Tespit Tablosu")

        warn_str_video = gr.Textbox(label="Warning Notes")

        video_button = gr.Button("Video İşle")
        video_button.click(
            fn=detect_from_video,
            inputs=video_input,
            outputs=[video_output, video_table_output, warn_str_video]
        )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a7992b2fb0cd8488c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
